#Previsione Livelli Ricoveri Terapia Intensiva, Nuovi Positivi e Deceduti per COVID

Nel seguente notebook sono stati utilizzati tre diversi modelli, LSTM, XGBoost e ARIMA con i dati resi disponibili dal Ministero della Salute per effettuare un forecast con orizzonte 1, 2, 7 e 14 giorni dei valori considerati.

Sono stati usati i dati regionali resi disponibili via file .csv al [link](https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv) e filtrati per regione di interesse, in questo caso Emilia-Romagna.
Ogni riga è un aggiornamento giornaliero dei seguenti valori: 
data, stato, codice_regione, denominazione_regione, lat, long, ricoverati_con_sintomi, terapia_intensiva,totale_ospedalizzati, isolamento_domiciliare, totale_positivi, variazione_totale_positivi, nuovi_positivi, dimessi_guariti, deceduti, casi_da_sospetto_diagnostico, casi_da_screening, totale_casi, tamponi, casi_testati, note, ingressi_terapia_intensiva, note_test, note_casi, totale_positivi_test_molecolare, totale_positivi_test_antigenico_rapido, tamponi_test_molecolare, tamponi_test_antigenico_rapido, codice_nuts_1, codice_nuts_2. 

Di questi sono state poi utilizzate per l'analisi: 
data, ricoverati_con_sintomi, terapia_intensiva, totale_ospedalizzati, variazione_totale_positivi, nuovi_positivi, deceduti, tamponi, ingressi_terapia_intensiva.

I valori di deceduti e tamponi sono stati differenziati per avere l'incremento giornaliero invece del dato cumulativo che era presente.

L'analisi svolta è su singola variabile.

I dati sono stati divisi con percentuale di 80 e 20 rispettivamente per training e testing.

Il primo 80% di dati in fase di tuning è a sua volta divisa in 80% di train e 20% di validation, in modo che il tuning non venga effettuato su dati di test.

Dopo la suddivisione è stato effettuato lo scaling attravero un MinMaxScaler che comprime i valori tra -1 ed 1 (la trasformazione viene poi invertita dopo la previsione).

Per i modelli XGBoost e LSTM sono stati preparati i dati creando per ogni giorno un array di lag temporali e di valori futuri della variabile presa in considerazione:
(t-n,...,t-1,t) -> (t+1,...,t+p) con n = giorni di lag e p = orizzonte di previsione.
Questi valori sono stati utilizzati per il training dei due modelli utilizzando le variabili lag come ingresso del modello e i valori futuri come target.

L'errore considerato è il Mean Absolute Error sull'orizzonte di previsione. Viene poi considerata la media dei MAE calcolati su tutte le previsioni.

Gli iper-parametri sono stati scelti attraverso una grid search per ogni variabile e per ogni orizzonte temporale selezionando il modello che da una media dei MAE inferiore.

Il grafico mostra l'ultimo slot temporale di previsione.


In [7]:
import os

if 'google.colab' in str(get_ipython()):
    files = ['util.py',
             'windows.py',
             'models']

    for file in files:
        os.system('rm ./' + file)
        os.system(
            'wget -nv https://raw.githubusercontent.com/marco-mazzoli/progetto-tesi/master/' + file)

import pandas as pd
import numpy as np
import warnings
from matplotlib import pyplot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.arima_model import ARIMA
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from numpy.random import seed
import plotly.graph_objects as go

# fix for 'package not found' when installing in Anaconda environment
if 'google.colab' not in str(get_ipython()):
    import pip
    pip.main(['install', 'xgboost'])

from xgboost import XGBRegressor
from util import select_relevant_rows, select_attributes, read_movement_data, download_updated_mobility_data, download_updated_mobility_data, save_config, load_config


In [8]:
use_existing_config = True
column_to_predict = 'terapia_intensiva'
columns = ['nuovi_positivi', 'terapia_intensiva', 'deceduti']
split_percent = 0.80
region_focus = 'Emilia-Romagna'
attribute_focus = 'denominazione_regione'
n_futures = [1, 2, 7, 14]

In [9]:
local_region_path = r'../COVID-19/dati-regioni/dpc-covid19-ita-regioni.csv'
remote_region_path = r'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

regions_frame = pd.read_csv(remote_region_path)

region_focus_data = select_relevant_rows(
    regions_frame,
    attribute_focus,
    region_focus
)

frame_interesting_columns = select_attributes(region_focus_data, [
    'data',
    'ricoverati_con_sintomi',
    'terapia_intensiva',
    'totale_ospedalizzati',
    'variazione_totale_positivi',
    'nuovi_positivi',
    'deceduti',
    'tamponi',
    'ingressi_terapia_intensiva'
])

frame_interesting_columns = pd.DataFrame(frame_interesting_columns)
frame_interesting_columns['data'] = pd.to_datetime(
    frame_interesting_columns['data'])
frame_interesting_columns['data'] = frame_interesting_columns['data'].dt.strftime(
    r'%Y-%m-%d')
frame_interesting_columns['data'] = pd.to_datetime(frame_interesting_columns['data'])
# frame_interesting_columns = frame_interesting_columns[frame_interesting_columns['data'] < pd.to_datetime('2022-01-31')]
frame_interesting_columns = frame_interesting_columns.fillna(0)

frame_interesting_columns.rename(columns={'data': 'date'}, inplace=True)
frame_interesting_columns.set_index('date', inplace=True)

# revert cumulative data
frame_interesting_columns['deceduti'] = frame_interesting_columns['deceduti'].diff(
)
frame_interesting_columns['tamponi'] = frame_interesting_columns['tamponi'].diff(
)
frame_interesting_columns.dropna(inplace=True)

frame_interesting_columns.tail

# numpy seed
seed(1)

In [10]:
for column in columns:
    frame = frame_interesting_columns[column]
    trace = go.Scatter(
        x=frame.index, y=frame, mode='lines', name='Prediction')
    layout = go.Layout(
        title=column, xaxis={'title': 'Date'},
        yaxis={'title': column}, autosize=False, 
            width=600, height=400)
    fig = go.Figure(
        data=[trace], layout=layout)
    fig.show()

In [11]:
def split_series(series, n_past, n_future, arima=False):
    X, y, X_indexes, y_indexes = list(), list(), list(), list()
    index = np.array(series.index).reshape(series.values.shape[0], 1)
    series = series.values

    for window_start in range(len(series)):
        past_end = window_start + n_past
        future_end = past_end + n_future
        if future_end > len(series):
            break
        start = 0 if arima == True else window_start

        past, future = series[start:past_end,
                              :], series[past_end:future_end, :]
        past_index, future_index = index[start:past_end,
                                         :], index[past_end:future_end, :]
        X.append(past)
        y.append(future)
        X_indexes.append(past_index)
        y_indexes.append(future_index)

    return np.array(X), np.array(y), np.array(X_indexes), np.array(y_indexes)


def plot_last_results(
    results, column_to_predict):
        pred = results[-1][1][-1]['pred']
        test = results[-1][1][-1]['y_test']

        prediction_trace = go.Scatter(
            x=pred.index, y=pred, mode='lines', name='Prediction')
        truth_trace = go.Scatter(
            x=test.index, y=test, mode='lines', name='Ground Truth')
        layout = go.Layout(
            title=column_to_predict, xaxis={'title': 'Date'},
            yaxis={'title': column_to_predict}, autosize=False, 
            width=600, height=400)
        fig = go.Figure(
            data=[prediction_trace, truth_trace], layout=layout)
        fig.show()

def plot_mape(results, column_to_predict):
    mapes = list(map(lambda x:x[1][1], results))
    dates = list(map(lambda x:x[0], results))

    mapes_trace = go.Scatter(
        x=dates, y=mapes, mode='lines', name='Mape')
    layout = go.Layout(
        title='mape: ' + column_to_predict, xaxis={'title': 'Date'},
        yaxis={'title': 'mape'}, autosize=False, 
        width=600, height=400)
    fig = go.Figure(
        data=[mapes_trace], layout=layout)
    fig.show()


#LSTM Multi Output 

In [12]:
def define_compile_lstm(config, input_shape, n_future=7):
    model = Sequential()
    model.add(LSTM(config['input'], activation=config['activation'],
                   input_shape=input_shape, return_sequences=True))
    model.add(LSTM(config['hidden'], activation=config['activation'],
                   return_sequences=False))
    model.add(Dropout(config['dropout']))
    model.add(Dense(n_future))
    model.compile(optimizer=config['optimizer'], loss=config['loss'])
    return model


def define_lstm_configs():
    input = [32, 64, 128]
    hidden = [32, 64, 128]
    activation = ['relu']
    dropout = [0.1, 0.2]
    out = [1]
    optimizer = ['adam']
    loss = ['mae']
    look_back = [28]

    configs = []
    keys = ['input', 'hidden', 'activation', 'dropout', 'out',
            'optimizer', 'loss', 'look_back']

    for i in input:
        for j in hidden:
            for k in activation:
                for l in dropout:
                    for m in out:
                        for n in optimizer:
                            for o in loss:
                                for p in look_back:
                                        config = dict(
                                            zip(keys, (i, j, k, l, m, n, o, p)))
                                        configs.append(config)

    return configs


def execute_lstm(
        dataframe, column_to_predict, config, split_percent=0.80, multi=False, n_future=7):
    df = dataframe[column_to_predict].copy()

    split = int(split_percent*len(df))

    n_past = config['look_back']
    n_features = 1

    train, test = pd.DataFrame(df[:split]), pd.DataFrame(df[split:])

    scalers = {}

    for i in train.columns:
        scaler = MinMaxScaler(feature_range=(-1, 1))
        s_s = scaler.fit_transform(train[i].values.reshape(-1, 1))
        s_s = np.reshape(s_s, len(s_s))
        scalers['scaler_' + i] = scaler
        train[i] = s_s

    for i in test.columns:
        scaler = scalers['scaler_'+i]
        s_s = scaler.transform(test[i].values.reshape(-1, 1))
        s_s = np.reshape(s_s, len(s_s))
        scalers['scaler_'+i] = scaler
        test[i] = s_s

    X_train, y_train, X_train_indexes, y_train_indexes = split_series(
        train, n_past, n_future)
    X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], n_features))
    y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))

    X_test, y_test, X_test_indexes, y_test_indexes = split_series(
        test, n_past, n_future)
    X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], n_features))
    y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))

    model = define_compile_lstm(config, input_shape=(n_past,n_features), n_future=n_future)

    history = model.fit(
        X_train, y_train, epochs=50, validation_data=(X_test, y_test),
        verbose=0, shuffle=False)

    pred = model.predict(X_test)

    pred = pred.reshape(pred.shape[0], pred.shape[1], 1)

    for index, i in enumerate(train.columns):
        scaler = scalers['scaler_'+i]

        pred[:, :, index] = scaler.inverse_transform(pred[:, :, index])

        y_train[:, :, index] = scaler.inverse_transform(y_train[:, :, index])
        y_test[:, :, index] = scaler.inverse_transform(y_test[:, :, index])

    df_results = []

    for i in range(len(y_test)):
        current = pd.DataFrame(
            {'y_test':y_test[i].reshape(-1),
            'pred':pred[i].reshape(-1),
            'dates':y_test_indexes[i].reshape(-1)})
        current.set_index('dates', inplace=True)
        df_results.append(current)

    results = {}

    for el in df_results:
        mae = mean_absolute_error(el['y_test'], el['pred'])
        mape = mean_absolute_percentage_error(el['y_test'], el['pred'])
        results[el.index[0]] = (mae, mape, el)

    return list(results.items())


def grid_search_lstm(
        dataframe, column_to_predict, split_percent=0.80, multi=False, n_future=7):
    configs = define_lstm_configs()
    results = []

    split = int(split_percent*len(dataframe))
    df_val = dataframe[:split].copy()

    for config in configs:
        sorted_results = execute_lstm(
            df_val, column_to_predict, config, split_percent=split_percent,
            multi=multi, n_future=n_future)

        results.append((np.mean(np.array(list(map(lambda x:x[1][0], sorted_results)))), config))

    return configs_results

summaries = []

for n_future in n_futures:
    for column_to_predict in columns:
        config_path = region_focus + '_' + 'uni_lstm_config' + '_' + column_to_predict
        config_path = config_path + '_' + str(n_future)
        if use_existing_config:
            if not os.path.isfile(config_path):
                os.system('wget -nv https://raw.githubusercontent.com/marco-mazzoli/progetto-tesi/master/configs/' + config_path)
            config = load_config(config_path)
            results = execute_lstm(
                frame_interesting_columns, column_to_predict=column_to_predict,
                config=config, split_percent=split_percent, multi=False, n_future=n_future)
            os.system('rm ' + config_path)
        else:
            configs_results = grid_search_lstm(
                frame_interesting_columns, column_to_predict, split_percent=split_percent,
                multi=False, n_future=n_future)

            configs_results.sort()
            config = configs_results[0][-1]

            save_config(config_path, config)

            results = execute_lstm(
                frame_interesting_columns, column_to_predict=column_to_predict,
                config=config, split_percent=split_percent, multi=False, n_future=n_future)
        
        avg_mae = np.mean(np.array(list(map(lambda x:x[1][0], results))))
        avg_mape = np.mean(np.array(list(map(lambda x:x[1][1], results))))

        summary = '|' + column_to_predict + '| seq len ' + str(n_future) + '| mae: ' + str(avg_mae) + '| mape: ' + str(avg_mape) + '| best config: ' + str(config)

        print(summary)
        summaries.append(summary)

        if n_future > 2:
            plot_last_results(results, column_to_predict)
        else:
            print(('Pred: ', results[-1][1][-1]['pred'].values))
            print(('Test: ', results[-1][1][-1]['y_test'].values))
        plot_mape(results, column_to_predict)
        
print(summaries)


|nuovi_positivi| seq len 1| mae: 1965105.1640391473| mape: 223.78151613058824| best config: {'input': 128, 'hidden': 128, 'activation': 'relu', 'dropout': 0.1, 'out': 1, 'optimizer': 'adam', 'loss': 'mae', 'look_back': 28}
('Pred: ', array([20598084.], dtype=float32))
('Test: ', array([4554.]))


|terapia_intensiva| seq len 1| mae: 6.949281858361283| mape: 0.08637542338798004| best config: {'input': 128, 'hidden': 32, 'activation': 'relu', 'dropout': 0.2, 'out': 1, 'optimizer': 'adam', 'loss': 'mae', 'look_back': 28}
('Pred: ', array([147.35931], dtype=float32))
('Test: ', array([140.]))


|deceduti| seq len 1| mae: 4.102243701271389| mape: 530336025890282.06| best config: {'input': 128, 'hidden': 32, 'activation': 'relu', 'dropout': 0.2, 'out': 1, 'optimizer': 'adam', 'loss': 'mae', 'look_back': 28}
('Pred: ', array([36.87207], dtype=float32))
('Test: ', array([32.]))


|nuovi_positivi| seq len 2| mae: 78546.00803214626| mape: 7.14125125089689| best config: {'input': 64, 'hidden': 128, 'activation': 'relu', 'dropout': 0.1, 'out': 1, 'optimizer': 'adam', 'loss': 'mae', 'look_back': 28}
('Pred: ', array([666133.06, 359518.34], dtype=float32))
('Test: ', array([5203., 4554.]))


|terapia_intensiva| seq len 2| mae: 5.458125357042277| mape: 0.06761901931485682| best config: {'input': 128, 'hidden': 128, 'activation': 'relu', 'dropout': 0.1, 'out': 1, 'optimizer': 'adam', 'loss': 'mae', 'look_back': 28}
('Pred: ', array([155.8999 , 157.69351], dtype=float32))
('Test: ', array([144., 140.]))


...

|deceduti| seq len 7| mae: 4.759721855973511| mape: 464450158539090.4| best config: {'input': 128, 'hidden': 32, 'activation': 'relu', 'dropout': 0.1, 'out': 1, 'optimizer': 'adam', 'loss': 'mae', 'look_back': 28}


|nuovi_positivi| seq len 14| mae: 42911.58805161163| mape: 3.52742629709144| best config: {'input': 64, 'hidden': 64, 'activation': 'relu', 'dropout': 0.2, 'out': 1, 'optimizer': 'adam', 'loss': 'mae', 'look_back': 28}


|terapia_intensiva| seq len 14| mae: 16.8564652314707| mape: 0.17214965118088152| best config: {'input': 64, 'hidden': 32, 'activation': 'relu', 'dropout': 0.1, 'out': 1, 'optimizer': 'adam', 'loss': 'mae', 'look_back': 28}


|deceduti| seq len 14| mae: 5.017400140521907| mape: 467587266646693.0| best config: {'input': 32, 'hidden': 32, 'activation': 'relu', 'dropout': 0.1, 'out': 1, 'optimizer': 'adam', 'loss': 'mae', 'look_back': 28}


["|nuovi_positivi| seq len 1| mae: 1965105.1640391473| mape: 223.78151613058824| best config: {'input': 128, 'hidden': 128, 'activation': 'relu', 'dropout': 0.1, 'out': 1, 'optimizer': 'adam', 'loss': 'mae', 'look_back': 28}", "|terapia_intensiva| seq len 1| mae: 6.949281858361283| mape: 0.08637542338798004| best config: {'input': 128, 'hidden': 32, 'activation': 'relu', 'dropout': 0.2, 'out': 1, 'optimizer': 'adam', 'loss': 'mae', 'look_back': 28}", "|deceduti| seq len 1| mae: 4.102243701271389| mape: 530336025890282.06| best config: {'input': 128, 'hidden': 32, 'activation': 'relu', 'dropout': 0.2, 'out': 1, 'optimizer': 'adam', 'loss': 'mae', 'look_back': 28}", "|nuovi_positivi| seq len 2| mae: 78546.00803214626| mape: 7.14125125089689| best config: {'input': 64, 'hidden': 128, 'activation': 'relu', 'dropout': 0.1, 'out': 1, 'optimizer': 'adam', 'loss': 'mae', 'look_back': 28}", "|terapia_intensiva| seq len 2| mae: 5.458125357042277| mape: 0.06761901931485682| best config: {'input':

#XGBoost Multi Output

In [13]:
def define_xgb_configs():
    max_depth = [3, 6, 10]
    learning_rate = [0.01, 0.05, 0.1]
    n_estimators = [50, 100, 500, 1000]
    colsample_bytree = [0.3, 0.7]
    look_back = [28]
    objectives = ['reg:squarederror']

    configs = []
    keys = ['max_depth', 'learning_rate', 'n_estimators', 'colsample_bytree',
            'look_back', 'objective']

    for i in max_depth:
        for j in learning_rate:
            for k in n_estimators:
                for l in colsample_bytree:
                    for m in look_back:
                            for n in objectives:
                                config = dict(
                                    zip(keys, (i, j, k, l, m, n)))
                            configs.append(config)

    return configs


def execute_xgb(
        dataframe, column_to_predict, config, split_percent=0.80, n_future=7, multi=False):
    df = dataframe[column_to_predict].copy()
    split = int(split_percent*len(df))

    n_past = config['look_back']
    n_features = 1

    train, test = pd.DataFrame(df[:split]), pd.DataFrame(df[split:])

    scalers = {}

    for i in train.columns:
        scaler = MinMaxScaler(feature_range=(-1, 1))
        s_s = scaler.fit_transform(train[i].values.reshape(-1, 1))
        s_s = np.reshape(s_s, len(s_s))
        scalers['scaler_' + i] = scaler
        train[i] = s_s

    for i in test.columns:
        scaler = scalers['scaler_'+i]
        s_s = scaler.transform(test[i].values.reshape(-1, 1))
        s_s = np.reshape(s_s, len(s_s))
        scalers['scaler_'+i] = scaler
        test[i] = s_s

    X_train, y_train, X_train_indexes, y_train_indexes = split_series(
        train, n_past, n_future)

    X_test, y_test, X_test_indexes, y_test_indexes = split_series(
        test, n_past, n_future)

    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1])
    y_train = y_train.reshape(y_train.shape[0], y_train.shape[1])
    X_train_indexes = X_train_indexes.reshape(
        X_train_indexes.shape[0], X_train_indexes.shape[1])
    y_train_indexes = y_train_indexes.reshape(
        y_train_indexes.shape[0], y_train_indexes.shape[1])

    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1])
    y_test = y_test.reshape(y_test.shape[0], y_test.shape[1])
    X_test_indexes = X_test_indexes.reshape(
        X_test_indexes.shape[0], X_test_indexes.shape[1])
    y_test_indexes = y_test_indexes.reshape(
        y_test_indexes.shape[0], y_test_indexes.shape[1])

    model = define_xgb(config)
    warnings.filterwarnings(action='ignore', category=UserWarning)
    model.fit(X_train, y_train)

    prediction = model.predict(X_test)  

    prediction[:, :] = scaler.inverse_transform(prediction[:, :])
    y_test[:, :] = scaler.inverse_transform(y_test[:, :])

    df_results = []

    for i in range(len(y_test)):
        current = pd.DataFrame(
            {'y_test':y_test[i].reshape(-1),
            'pred':prediction[i].reshape(-1),
            'dates':y_test_indexes[i].reshape(-1)})
        current.set_index('dates', inplace=True)
        df_results.append(current)

    results = {}

    for el in df_results:
        mae = mean_absolute_error(el['y_test'], el['pred'])
        mape = mean_absolute_percentage_error(el['y_test'], el['pred'])
        results[el.index[0]] = (mae, mape, el)

    return list(results.items())


def grid_search_xgb(
        dataframe, column_to_predict, split_percent=0.80, n_future=7, multi=False):
    configs = define_xgb_configs()
    results = []

    split = int(split_percent*len(dataframe))
    df_val = dataframe[:split].copy()

    for config in configs:
        results = execute_xgb(
            df_val, column_to_predict, config, split_percent=split_percent,
            multi=multi, n_future=n_future)

        results.append((np.mean(np.array(list(map(lambda x:x[1][0], results)))), config))

    return configs_results


def define_xgb(config):
    return MultiOutputRegressor(XGBRegressor(**config, verbosity=0))

summaries = []

for n_future in n_futures:
    for column_to_predict in columns:
        config_path = region_focus + '_' + 'uni_xgb_config' + '_' + column_to_predict
        config_path = config_path + '_' + str(n_future)
        if use_existing_config:
            if not os.path.isfile(config_path):
                os.system('wget -nv https://raw.githubusercontent.com/marco-mazzoli/progetto-tesi/master/configs/' + config_path)
            config = load_config(config_path)
            results = execute_xgb(
                frame_interesting_columns, split_percent=split_percent, config=config,
                column_to_predict=column_to_predict, multi=False, n_future=n_future)
            os.system('rm ' + config_path)
        else:
            configs_results = grid_search_xgb(
                frame_interesting_columns, split_percent=split_percent,
                column_to_predict=column_to_predict, multi=False)

            configs_results.sort(key=lambda tup: tup[0])

            config = configs_results[0][-1]

            save_config(config_path, config)

            results = execute_xgb(
                frame_interesting_columns, split_percent=split_percent, config=config,
                column_to_predict=column_to_predict, multi=False, n_future=n_future)

        avg_mae = np.mean(np.array(list(map(lambda x:x[1][0], results))))
        avg_mape = np.mean(np.array(list(map(lambda x:x[1][1], results))))

        summary = '|' + column_to_predict + '| seq len ' + str(n_future) + '| mae: ' + str(avg_mae) + '| mape: ' + str(avg_mape) + '| best config: ' + str(config)

        print(summary)
        summaries.append(summary)

        if n_future > 2:
            plot_last_results(results, column_to_predict)
        else:
            print(('Pred: ', results[-1][1][-1]['pred'].values))
            print(('Test: ', results[-1][1][-1]['y_test'].values))

        plot_mape(results, column_to_predict)

print(summaries)

|nuovi_positivi| seq len 1| mae: 4348.477621061906| mape: 0.35797491300150086| best config: {'max_depth': 3, 'learning_rate': 0.01, 'n_estimators': 500, 'colsample_bytree': 0.7, 'look_back': 28, 'objective': 'reg:squarederror'}
('Pred: ', array([2611.736], dtype=float32))
('Test: ', array([4554.]))


|terapia_intensiva| seq len 1| mae: 7.563724484650983| mape: 0.0848032733241689| best config: {'max_depth': 3, 'learning_rate': 0.01, 'n_estimators': 1000, 'colsample_bytree': 0.3, 'look_back': 28, 'objective': 'reg:squarederror'}
('Pred: ', array([130.19366], dtype=float32))
('Test: ', array([140.]))


|deceduti| seq len 1| mae: 4.360760958298393| mape: 335113646822836.7| best config: {'max_depth': 3, 'learning_rate': 0.1, 'n_estimators': 50, 'colsample_bytree': 0.3, 'look_back': 28, 'objective': 'reg:squarederror'}
('Pred: ', array([40.853657], dtype=float32))
('Test: ', array([32.]))


|nuovi_positivi| seq len 2| mae: 4425.404572871694| mape: 0.3744485299677831| best config: {'max_depth': 3, 'learning_rate': 0.01, 'n_estimators': 500, 'colsample_bytree': 0.7, 'look_back': 28, 'objective': 'reg:squarederror'}
('Pred: ', array([2619.184 , 2417.9436], dtype=float32))
('Test: ', array([5203., 4554.]))


|terapia_intensiva| seq len 2| mae: 8.064177831013996| mape: 0.09131613429341422| best config: {'max_depth': 3, 'learning_rate': 0.01, 'n_estimators': 1000, 'colsample_bytree': 0.3, 'look_back': 28, 'objective': 'reg:squarederror'}
('Pred: ', array([129.28381, 127.52169], dtype=float32))
('Test: ', array([144., 140.]))


...

|deceduti| seq len 7| mae: 4.69934258204769| mape: 424647249940485.6| best config: {'max_depth': 3, 'learning_rate': 0.1, 'n_estimators': 50, 'colsample_bytree': 0.3, 'look_back': 28, 'objective': 'reg:squarederror'}


|nuovi_positivi| seq len 14| mae: 4749.556054459901| mape: 0.4367675005356777| best config: {'max_depth': 3, 'learning_rate': 0.01, 'n_estimators': 500, 'colsample_bytree': 0.7, 'look_back': 28, 'objective': 'reg:squarederror'}


|terapia_intensiva| seq len 14| mae: 19.316237643319354| mape: 0.24098190944432596| best config: {'max_depth': 3, 'learning_rate': 0.01, 'n_estimators': 1000, 'colsample_bytree': 0.3, 'look_back': 28, 'objective': 'reg:squarederror'}


|deceduti| seq len 14| mae: 5.396112342007687| mape: 452510857695909.2| best config: {'max_depth': 3, 'learning_rate': 0.1, 'n_estimators': 50, 'colsample_bytree': 0.3, 'look_back': 28, 'objective': 'reg:squarederror'}


["|nuovi_positivi| seq len 1| mae: 4348.477621061906| mape: 0.35797491300150086| best config: {'max_depth': 3, 'learning_rate': 0.01, 'n_estimators': 500, 'colsample_bytree': 0.7, 'look_back': 28, 'objective': 'reg:squarederror'}", "|terapia_intensiva| seq len 1| mae: 7.563724484650983| mape: 0.0848032733241689| best config: {'max_depth': 3, 'learning_rate': 0.01, 'n_estimators': 1000, 'colsample_bytree': 0.3, 'look_back': 28, 'objective': 'reg:squarederror'}", "|deceduti| seq len 1| mae: 4.360760958298393| mape: 335113646822836.7| best config: {'max_depth': 3, 'learning_rate': 0.1, 'n_estimators': 50, 'colsample_bytree': 0.3, 'look_back': 28, 'objective': 'reg:squarederror'}", "|nuovi_positivi| seq len 2| mae: 4425.404572871694| mape: 0.3744485299677831| best config: {'max_depth': 3, 'learning_rate': 0.01, 'n_estimators': 500, 'colsample_bytree': 0.7, 'look_back': 28, 'objective': 'reg:squarederror'}", "|terapia_intensiva| seq len 2| mae: 8.064177831013996| mape: 0.09131613429341422| 